In [1]:
import pandas as pd

In [2]:
df1 = pd.DataFrame({'day_of_week': [0,1,2,3,4,5,6],
                    'day_spend_cent': ['0.148', '0.158', '0.147','0.146', '0.143', '0.128', '0.127']})

df2 = pd.DataFrame({'week_bin': [1,2,3,4,5],
                    'week_spend_cent': ['.2', '.25', '.05', '.1', '.3']})

### want to combine df1 & df2
#### each day_of_week applied to each week_bin & multiplied with output in 3rd column


In [3]:
df1.head()

,day_of_week,day_spend_cent
0,0,0.148
1,1,0.158
2,2,0.147
3,3,0.146
4,4,0.143


In [4]:
df2.head()

,week_bin,week_spend_cent
0,1,.2
1,2,.25
2,3,.05
3,4,.1
4,5,.3


**desired solution**

|bin | day | spend |
| --- | --- | --- |
| 0 | 0  | 0.148 x 0.2 |
| 0 | 1  | 0.158 x 0.2 |
| 0 | 2  | 0.147 x 0.2 |
| 0 | 3  | . |
| 0 | 4  | . |
| 0 | 5  | . |
| 0 | 6  | . |
| 1 | 0  | 0.148 x 0.25 |

In [5]:
# docs for multiindexing:
# https://pandas.pydata.org/docs/user_guide/advanced.html
# generate a 2-level index
bin_index = df2['week_bin'].values
day_of_week_index = df1['day_of_week'].values

df3_index = pd.MultiIndex.from_product([bin_index, day_of_week_index],
                                        names = ['week_bin', 'day_of_week'])

In [6]:
# initialize dataframe wanted and create col name 'spend'
df3 = pd.DataFrame(index=df3_index, columns=['spend'])

# loop through each record
# TODO: it would look cooler if we could figure out how to vectorize this, but this will work just fine
# good article on iterating through dataframes:
# https://towardsdatascience.com/heres-the-most-efficient-way-to-iterate-through-your-pandas-dataframe-4dad88ac92ee
# itertuples() is the most efficient method that still returns an index
for row in df3.itertuples():
    
    # get the index
    index = row.Index
    week_bin, day_of_week = index  #makes 2 indexes
    
    # look up values in df1 and df2
    spend_cent = float(df1[df1['day_of_week']==day_of_week]['day_spend_cent'].values[0])
    Name = float(df2[df2['week_bin']==week_bin]['week_spend_cent'].values[0])
    
    # assign value
    df3.loc[index, 'spend'] = spend_cent * Name


In [7]:
df3

spend
week_bin day_of_week         
1        0             0.0296
         1             0.0316
         2             0.0294
         3             0.0292
         4             0.0286
         5             0.0256
         6             0.0254
2        0              0.037
         1             0.0395
         2            0.03675
         3             0.0365
         4            0.03575
         5              0.032
         6            0.03175
3        0             0.0074
         1             0.0079
         2            0.00735
         3             0.0073
         4            0.00715
         5             0.0064
         6            0.00635
4        0             0.0148
         1             0.0158
         2             0.0147
         3             0.0146
         4             0.0143
         5             0.0128
         6             0.0127
5        0             0.0444
         1             0.0474
         2             0.0441
         3             0.0438
         4             0.0429
         5             0.0384
         6             0.0381

- assume your percent of budget spent on week 1 for March, April, May were 19%, 17%, and 18%, so an average of 18% spent in week one.
- Say our budget for June is 100k. Then the budget for week 1 is 100k * 0.18 = $18k.
- Say our day parting percents are 15%, 16%, 15%, 15%, 14%, 14%, 11% (Mon-Sun)
- Say June 1 is a Wed (which it was). Then the spend on June 1 is 18k*0.15 (the 15% for Wed). Spend June 2 (Thurs) is 18k * 0.15,  and so on for the rest of the week.